In [115]:
class Edge:
    def __init__(self, frm, to):#UNDIRECTED graph
        if frm < to:
            self.frm = frm # from
            self.to = to
        else:
            self.to = frm
            self.frm = to
    def __hash__(self):
        return hash((self.frm, self.to))
    def __eq__(self, other):
        return isinstance(self, type(other)) and other.frm == self.frm and self.to == other.to

edges = set()#[]

test_set = set()

with open("dataset.txt") as f:
    for line in f:
        if line[0] == "%":
            continue
        content = line.split()
        if content[0] != content[1]:# and Edge(content[0], content[1]) not in test_set:
            #edges += [Edge(content[0], content[1])]
            test_set.add(Edge(content[0], content[1]))
        
#check unique edges

In [102]:
print(len(edges))
print(Edge('1','2').__eq__(Edge('2','1')))
print(Edge('1','2').__hash__())
print(Edge('2','1').__hash__())

2408
True
-3134134563166582134
-3134134563166582134


In [116]:
import random

class Triest_base:
    def __init__(self, set_limit=200):
        self.t = 0
        self.tau = 0
        self.edge_sampled = set()
        self.set_limit = set_limit #>=6
        self.counters = {}

    def sample(self, edge):
        if self.t<=self.set_limit:
            return True
        if random.random() <= (self.set_limit/self.t):
            remove_el = random.sample(self.edge_sampled, 1)[0]
            self.edge_sampled.remove(remove_el)
            self.update_counters('-', edge)
            return True
        return False

    def update_counters(self, op, edge):    
        s1 = set()
        s2 = set()
        for e in self.edge_sampled:
            if e.frm == edge.frm:
                s1.add(e.to)
            if e.to == edge.frm:
                s1.add(e.frm)
            if e.frm == edge.to:
                s2.add(e.to)
            if e.to == edge.to:
                s2.add(e.frm)
        for c in (s1 & s2):
            if op == '+':
                self.tau+=1
                self.counters[c] = self.counters.get(c, 0)+1
                self.counters[edge.frm] = self.counters.get(edge.frm, 0)+1
                self.counters[edge.to] = self.counters.get(edge.to, 0)+1

            elif op == '-':
                self.tau-=1
                self.counters[c] = self.counters.get(c, 0)-1
                if self.counters[c] <= 0:
                    del self.counters[c]
                self.counters[edge.frm] = self.counters.get(edge.frm, 0)-1
                if self.counters[edge.frm] <= 0:
                    del self.counters[edge.frm]
                self.counters[edge.to] = self.counters.get(edge.to, 0)-1
                if self.counters[edge.to] <= 0:
                    del self.counters[edge.to]
                    
    def run(self, stream_edges):
        for edge in stream_edges:
            if self.t % 1000 == 0:
                print("element ", self.t, "value: ", self.tau)
            self.t += 1
            if self.sample(edge):
                self.edge_sampled.add(edge)
                self.update_counters('+', edge)
        eps = (self.t*(self.t-1)*(self.t-2))/(self.set_limit*(self.set_limit-1)*(self.set_limit-2))
        if eps < 1:
            eps = 1
        return self.tau*eps

In [ ]:
sample_size = 1000
t = Triest_base(sample_size)
exp = t.run(test_set)
t_val = Triest_base(len(test_set)).run(test_set)
print("With ", sample_size, "samples the expected value is ", exp," . The true value is ", t_val," . Error: ", abs(t_val - exp)," triangles")

element  0 value:  0
element  1000 value:  1
element  2000 value:  1
element  3000 value:  1
element  4000 value:  1
element  5000 value:  1
element  6000 value:  1
element  0 value:  0
element  1000 value:  1
element  2000 value:  17


In [ ]:
#V = set() #
#E = set() #set of edges